In [1]:
from pyquery import PyQuery as pq
import time
import requests
from selenium import webdriver
import csv
import re

In [2]:

def PCHome24(url):
    ###########
    # 開始爬蟲 #
    ###########
    res=requests.get(url)
    
    driver.get(res.url)
    html = driver.find_element_by_css_selector("*").get_attribute("outerHTML")
    doc = pq(html)
    # driver.quit()

    productList=[]
    doc.make_links_absolute(base_url=res.url)

    def addItemData():
        itemData={}    
        itemData['Name']=itemDoc('div.prod_info h5').text()
        itemData['Date']=time.strftime("%Y-%m-%d", time.localtime())
        itemData['Price']=itemDoc('div.prod_info .price_box .value').text()
        itemData['Path']=itemDoc('div.prod_info h5 a').attr('href')
        if re.match("(Kingston|金士頓|Micron|美光|Transcend|創見|ADATA|威剛|Apacer|宇瞻)",itemData['Name']):
            nameList=[i['Name'] for i in productList]
            if itemData['Name'] not in nameList:
                productList.append(itemData)

    #網頁商品展示頁面 上方 右邊
    for itemDoc in doc('#Block2Container > div.left > div').items():
        addItemData()

    #網頁商品展示頁面 上方 左邊
    for itemDoc in doc('#Block2Container > div.right dd').items():
        addItemData()

    #網頁商品展示頁面 下方 
    nowPage=doc
    while 1:
        for itemDoc in nowPage('#ProdGridContainer > dd').items():
            addItemData()

        if nowPage('#PaginationContainer > ul > li.sp > a').text() == '下一頁':
            pageLink=nowPage('#PaginationContainer > ul > li.sp > a').attr('href')
            driver.get(pageLink)
            html = driver.find_element_by_css_selector("*").get_attribute("outerHTML")     
            nowPage = pq(html)
            nowPage.make_links_absolute(base_url=pageLink)
        else:
            break

    ###############
    # 讀取過去資料 #
    ###############
    content=[]
    try:
        fIn=open("Price_Record.csv",'r',newline='')
        csvIn = csv.DictReader(fIn)
        for line in csvIn:
            content.append(dict(line))
        fIn.close()
    except FileNotFoundError: 
        with open("Price_Record.csv",'w',newline='') as fObj:
            fieldnames = ['Name','Date', 'Price','Path']
            csvOut = csv.DictWriter(fObj,fieldnames=fieldnames)
            csvOut.writeheader()       

    for i in productList:
        content.append(i)

    ###############
    # 排序商品名稱 #
    ###############
    def itemNameOrder(item):
        if re.match("(Kingston|金士頓)", item['Name']):
            return ('1'+item['Name'])
        elif re.match("(Micron|美光)", item['Name']):
            return ('2'+item['Name'])
        elif re.match("(Transcend|創見)", item['Name']):
            return ('3'+item['Name'])
        elif re.match("(ADATA|威剛)", item['Name']):
            return ('4'+item['Name'])
        elif re.match("(Apacer|宇瞻)", item['Name']):
            return ('5'+item['Name'])

    test=sorted(content,key=itemNameOrder)

    ###############
    # 存入商品紀錄 #
    ###############
    with open("Price_Record.csv",'w',newline='') as fObj:
        fieldnames = ['Name','Date', 'Price','Path']
        csvOut = csv.DictWriter(fObj,fieldnames=fieldnames)
        csvOut.writeheader()
        csvOut.writerows(test)

In [3]:
url=['https://24h.pchome.com.tw/store/DRAC5L','https://24h.pchome.com.tw/store/DRAC6X',\
    'https://24h.pchome.com.tw/store/DRAC6Q','https://24h.pchome.com.tw/store/DRAC78']

driver = webdriver.Chrome()

for i in url:
    PCHome24(i)
    
driver.quit()

In [4]:
# 已解決

# 1.網頁為動態 非靜態
# from selenium import webdriver
# driver = webdriver.Chrome()
# driver.get(res.url)
# html = driver.find_element_by_css_selector("*").get_attribute("outerHTML")
# doc = pq(html)

# 2.換頁時 無法讀取資料
# 每次換頁都要重新執行 selenium 的動作

# 3.如何選出想要的商品名稱
# import re
# if re.match("(Kingston|金士頓|Micron|美光|Transcend|創見|ADATA|威剛|Apacer|宇瞻)", itemData['Name']):

# 4.如何排列想要的商品順序
# sorted(content,key=itemNameOrder)
# 使用key參數 自訂函數

# 5.第一次建立資料 如何判斷 過去有存在紀錄
# 利用 try except
# 使用 FileNotFoundError

In [5]:
# 個人疑問

# 問題1
# 排序想要的名稱 如何處理?

# 問題2
# 若是忘記了是否曾經寫入 可能導致重複執行 資料重複

# 問題3
# 如何使程式每天自動執行

# 問題4
# telnet協定 pyquery可以爬取嗎?
# 還是只能web

# 問題5
# 動態網站 只能靠 seleniium?

# 問題6
# 爬蟲框架?